In [1]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt


In [2]:
def get_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Create model that maps input -> (conv outputs, predictions)
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Run forward + record gradients
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # Get gradient of class w.r.t conv outputs
    grads = tape.gradient(class_channel, conv_outputs)

    # Mean intensity of gradients per feature map
    weights = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    # Weighted sum
    heatmap = tf.reduce_sum(tf.multiply(weights, conv_outputs), axis=-1)

    # Normalize
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap = heatmap.numpy()

    return heatmap


In [3]:
def display_gradcam(org_img_path, heatmap, alpha=0.4):
    img = cv2.imread(org_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

    superimposed = heatmap_color * alpha + img

    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.title("Original")
    plt.imshow(img)
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.title("Grad-CAM")
    plt.imshow(superimposed.astype(np.uint8))
    plt.axis("off")
    plt.show()


In [4]:
img_path = "ChatGPT Image Nov 23, 2025, 02_52_12 AM.png"

# Prepare the image
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224,224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Name of the last conv layer for your model
last_conv_layer = "conv5_block3_out"   # مثال ResNet50

# Apply Grad-CAM
heatmap = get_gradcam_heatmap(img_array, model, last_conv_layer)

# Display
display_gradcam(img_path, heatmap)


NameError: name 'model' is not defined